# Geração de texto básico

De inicio, sera usado o GPT 2 porque:

* É gratuito
* Podemos usar a biblioteca transformers

Obs: GPT 2 só entende inglês

#### Instalação das bibliotecas

##### Instalação do pytorch com suporte a gpu de cuda versão 12.8 (verificar cuda da placa de vídeo)
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu128
##### Instalação do transformers
!pip install transformers

In [1]:
# Importando Bibliotecas
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence

In [2]:
# Inicializando o tokenizer e o modelo
tokenizer = AutoTokenizer.from_pretrained("gpt2")
model = AutoModelForCausalLM.from_pretrained("gpt2")

In [3]:
# Encoding o prompt para coletar input ids
prompt = "Hello boss ..."
input_ids = tokenizer.encode(prompt, return_tensors="pt") # pt = pytorch
#input_ids # Visualização

# Gerar texto usando o modelo
outputs = model.generate(input_ids, max_length = 100)
#outputs # Visualização
tokenizer.decode(outputs[0], skip_special_tokens=True)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


"Hello boss ... I'm sorry, but I'm not sure if you're ready to go. I'm not sure if you're ready to go. I'm not sure if you're ready to go. I'm not sure if you're ready to go. I'm not sure if you're ready to go. I'm not sure if you're ready to go. I'm not sure if you're ready to go. I'm not sure if you're ready to go. I'm not sure"

In [4]:
def gerar_texto_simples(prompt, model, tokenizer, max_length = 100):
    input_ids = tokenizer.encode(prompt, return_tensors="pt")
    outputs = model.generate(input_ids, max_length = max_length)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

In [5]:
# Testando a função
prompt = "Hello my friend ..."
texto_gerado = gerar_texto_simples(prompt,
                                  model,
                                  tokenizer,
                                  max_length = 100)
print(texto_gerado)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Hello my friend ... I'm not going to be able to do this. I'm not going to be able to do this. I'm not going to be able to do this. I'm not going to be able to do this. I'm not going to be able to do this. I'm not going to be able to do this. I'm not going to be able to do this. I'm not going to be able to do this. I'm not going to be able to


# Fine Tuning

In [6]:
dados = [
    "Machine Learning é uma área da inteligência artificial que permite que os computadores aprendam e tomem decisões sem serem explicitamente programados. O aprendizado é feito a partir de dados e o desempenho melhora com o tempo conforme novos dados são apresentados ao modelo.",
    "Existem três tipos principais de aprendizado de máquina: supervisionado, não supervisionado e por reforço. No aprendizado supervisionado, o modelo é treinado com dados rotulados, enquanto no não supervisionado, o modelo tenta encontrar padrões ocultos nos dados sem rótulos.",
    "Uma das aplicações mais conhecidas de Machine Learning é a classificação de e-mails como spam ou não spam. Isso é feito treinando um modelo com um grande conjunto de e-mails previamente classificados, permitindo que ele aprenda a distinguir padrões associados a cada categoria.",
    "Redes neurais artificiais são modelos inspirados no funcionamento do cérebro humano e são muito utilizadas em tarefas como reconhecimento de imagens, processamento de linguagem natural e tradução automática. Elas são compostas por camadas de neurônios artificiais que processam as informações de forma hierárquica.",
    "O treinamento de modelos de Machine Learning envolve a escolha de uma métrica de avaliação, como acurácia, precisão, recall ou F1-score, e a otimização de parâmetros para minimizar o erro da previsão. Técnicas como validação cruzada ajudam a evitar o overfitting.",
    "Machine Learning é amplamente usado em sistemas de recomendação, como os que vemos na Netflix ou Amazon, onde os algoritmos analisam o histórico de comportamento dos usuários para sugerir novos produtos ou conteúdos que sejam relevantes para cada perfil individual.",
    "Uma técnica muito usada para reduzir a dimensionalidade dos dados é a Análise de Componentes Principais (PCA), que transforma um conjunto de variáveis possivelmente correlacionadas em um conjunto menor de variáveis independentes chamadas de componentes principais.",
    "Modelos de aprendizado de máquina podem sofrer com o problema de overfitting, quando aprendem muito bem os dados de treino, mas não conseguem generalizar para dados novos. Uma das soluções para esse problema é o uso de regularização e aumento da quantidade de dados de treinamento.",
    "O pré-processamento de dados é uma etapa crucial em qualquer projeto de Machine Learning. Isso inclui tarefas como limpeza de dados, tratamento de valores ausentes, normalização e codificação de variáveis categóricas, garantindo que os dados estejam em um formato adequado para o treinamento do modelo.",
    "Com o crescimento de bibliotecas como Scikit-learn, TensorFlow e PyTorch, o desenvolvimento de modelos de Machine Learning se tornou mais acessível a desenvolvedores e cientistas de dados, permitindo a criação de soluções robustas em áreas como finanças, saúde, marketing e segurança."
]

In [7]:
data = [
    "Machine Learning is a field of artificial intelligence that enables computers to learn and make decisions without being explicitly programmed. Learning is done from data, and performance improves over time as new data is presented to the model.",
    "There are three main types of machine learning: supervised, unsupervised, and reinforcement learning. In supervised learning, the model is trained with labeled data, while in unsupervised learning, the model tries to find hidden patterns in unlabeled data.",
    "One of the most well-known applications of Machine Learning is classifying emails as spam or not spam. This is done by training a model with a large set of previously classified emails, allowing it to learn patterns associated with each category.",
    "Artificial neural networks are models inspired by the functioning of the human brain and are widely used in tasks such as image recognition, natural language processing, and machine translation. They consist of layers of artificial neurons that process information hierarchically.",
    "Training Machine Learning models involves choosing an evaluation metric, such as accuracy, precision, recall, or F1-score, and optimizing parameters to minimize prediction error. Techniques like cross-validation help avoid overfitting.",
    "Machine Learning is widely used in recommendation systems, such as those seen on Netflix or Amazon, where algorithms analyze users' behavior history to suggest new products or content that is relevant to each individual profile.",
    "A widely used technique to reduce data dimensionality is Principal Component Analysis (PCA), which transforms a set of possibly correlated variables into a smaller set of independent variables called principal components.",
    "Machine learning models can suffer from the problem of overfitting, where they learn the training data too well but fail to generalize to new data. One solution to this problem is the use of regularization and increasing the amount of training data.",
    "Data preprocessing is a crucial step in any Machine Learning project. This includes tasks such as data cleaning, handling missing values, normalization, and encoding categorical variables, ensuring that the data is in a suitable format for model training.",
    "With the growth of libraries like Scikit-learn, TensorFlow, and PyTorch, the development of Machine Learning models has become more accessible to developers and data scientists, allowing the creation of robust solutions in areas such as finance, healthcare, marketing, and security."
]

In [8]:
# Tokenização
# Todos os inputs precisam ter o mesmo tamanho (length)
# Add um dummy token no final que sera ignorado pelo sistema
# Ter o mesmo length => Isso é chamado de padding
tokenizer.pad_token = tokenizer.eos_token

In [9]:
# Tokenizando os dados
dados_tokenizados = [tokenizer.encode_plus(
    sentenca,
    add_special_tokens = True, #special tokens = pontuação e etc
    return_tensors = "pt", #pt = pytorch
    padding = "max_length",
    max_length = 60, #suficiente para caber todas as strings do projeto mas não sobrar muito
) for sentenca in data]
dados_tokenizados[:2] # exibindo apenas primeiros 2

[{'input_ids': tensor([[37573, 18252,   318,   257,  2214,   286, 11666,  4430,   326, 13536,
           9061,   284,  2193,   290,   787,  5370,  1231,   852, 11777, 27402,
             13, 18252,   318,  1760,   422,  1366,    11,   290,  2854, 19575,
            625,   640,   355,   649,  1366,   318,  5545,   284,   262,  2746,
             13, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
          50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
          1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0,
          0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])},
 {'input_ids': tensor([[ 1858,   389,  1115,  1388,  3858,   286,  4572,  4673,    25, 28679,
             11,   555, 16668, 16149,    11,   290, 37414,  4673,    13,   554,
          28679,  4673,    11,   262,  2746,   318,  8776,   351, 15494,  1366,
             11,  

In [10]:
# isolando os input ids e os attention mask
input_ids = [item['input_ids'].squeeze() for item in dados_tokenizados]
attention_masks = [item['attention_mask'].squeeze() for item in dados_tokenizados]

In [11]:
# converter os input ids e os attention masks para tensors
# esse passo é necessário para processar o tuned model
input_ids = torch.stack(input_ids)
attention_masks = torch.stack(attention_masks)

In [12]:
# padding todas as sequencias para se certificar que tenham o mesmo length / necessário
padded_input_ids = pad_sequence(input_ids,
                                batch_first = True,
                                padding_value = tokenizer.eos_token_id)
padded_attention_masks = pad_sequence(attention_masks,
                                     batch_first = True,
                                     padding_value = 0)

In [13]:
# Criando uma classe dataset customizada incluindo datalabels
class TextDataset(Dataset):
    def __init__(self, input_ids, attention_masks):
        self.input_ids = input_ids
        self.attention_masks = attention_masks
        self.labels = input_ids.clone()

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        return {
            'input_ids' : self.input_ids[idx],
            'attention_mask' : self.attention_masks[idx],
            'labels' : self.labels[idx]
        }

# criando objeto da classe
dataset = TextDataset(padded_input_ids, padded_attention_masks)
dataset[:2]

{'input_ids': tensor([[37573, 18252,   318,   257,  2214,   286, 11666,  4430,   326, 13536,
           9061,   284,  2193,   290,   787,  5370,  1231,   852, 11777, 27402,
             13, 18252,   318,  1760,   422,  1366,    11,   290,  2854, 19575,
            625,   640,   355,   649,  1366,   318,  5545,   284,   262,  2746,
             13, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
          50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256],
         [ 1858,   389,  1115,  1388,  3858,   286,  4572,  4673,    25, 28679,
             11,   555, 16668, 16149,    11,   290, 37414,  4673,    13,   554,
          28679,  4673,    11,   262,  2746,   318,  8776,   351, 15494,  1366,
             11,   981,   287,   555, 16668, 16149,  4673,    11,   262,  2746,
           8404,   284,  1064,  7104,  7572,   287,  9642,  9608,   276,  1366,
             13, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256]]),
 'attention_mask': tens

## Fine Tuning o GPT 2

In [14]:
# Preparando os dados em batches
dataloader = DataLoader(dataset, batch_size = 2, shuffle = True)

In [15]:
# Inicializando um optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr = 5e-5)

# Colocando o modelo em training mode
model.train()

# Loop de treino

# vezes que o modelo sera treinado
for epoch in range(10): 
    for batch in dataloader:
        # Pegando os input ids e attention mask
        input_ids = batch['input_ids']
        attention_mask = batch['attention_mask']

        # Resetando os gradients para zero / zerando o optimizer
        optimizer.zero_grad()

        # Forward pass
        # Processando o input e attention masks
        outputs = model(input_ids = input_ids,
                       attention_mask = attention_mask,
                       labels = input_ids)
        loss = outputs.loss

        # Backward pass: compute the gradients of the loss
        loss.backward()

        # Atualiza os parametros do modelo
        optimizer.step()
    
    # Printando o loss para a epoch atual para monitorar o progresso
    print(f"Epoch {epoch + 1} - Loss: {loss.item()}")

`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


Epoch 1 - Loss: 2.90610671043396
Epoch 2 - Loss: 2.357048988342285
Epoch 3 - Loss: 1.9948416948318481
Epoch 4 - Loss: 1.6582231521606445
Epoch 5 - Loss: 1.344558596611023
Epoch 6 - Loss: 0.9374242424964905
Epoch 7 - Loss: 0.7848874926567078
Epoch 8 - Loss: 0.4702298045158386
Epoch 9 - Loss: 0.4030720591545105
Epoch 10 - Loss: 0.4024461507797241


In [20]:
# Definindo função para gerar texto
def gerar_texto(prompt, model, tokenizer, max_length = 100):
    # Encoding the prompt to get the input ids
    inputs = tokenizer.encode_plus(prompt, return_tensors = "pt")
    input_ids = inputs['input_ids']
    attention_mask = inputs['attention_mask']

    # Gerar texto usando o modelo
    outputs = model.generate(input_ids, 
                             attention_mask = attention_mask, 
                             max_length = max_length)

    return tokenizer.decode(outputs[0], skip_special_tokens = True)

In [21]:
# Testando a função
prompt = "About Machine Learning, we "
texto_gerado = gerar_texto(prompt, model, tokenizer, max_length = 500)
print(texto_gerado)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


About Machine Learning, we  learn algorithms by training data, allowing each model with a large set of previously unstructured data.
